### NOTES

Initialization 

Weights distribution needs to be roughly unit gaussian

This is because of non-linear functions, which domain is (-1,1) (tanh) , (0,x)(ReLu), etc. Values higher or lower than STD of 1 are problematic because the gradien vanishes or causes dead neurons. E.g. 2 input values -30 and -40 have the same effect of outputing -1 in a tanh function, and given that the gradient of grad(tanh(-1)=0), there is no gradient broadcasted in the network.

So it is important to inialize weights properly.

But for bigger neural networks, this becomes intractable because it is difficult to compute the gains for weight initialization.

To address this problem, one can use Batch normalization

It is common to sprinkle Batch normalization into linear or convolutional layers

The stability of BN comes at a cost.BN couples forward propagation with the batch itself, because now the STD and MEAN of the batch are part of the forward pass. So there will be some noise in terms of the distribution of the weights

But this helps to regularize the net and prevents the weights of the net to overfit

BN also makes the input of the forward pass to depend on the STD and MEAN of the batch. So at inference time, we calculate the STD and MEAN of all the dataset (train, test or val) and use these fix values for inference forward pass. This can be done also during training time to avoid a second stage computation.

If using BN, it is better to not to have biases to the layer attached to BN